In [1]:
%run ../../main.py

In [2]:
from pyarc.algorithms import top_rules
from pyarc.data_structures import TransactionDB, Consequent, Antecedent, Item, ClassAssocationRule
from pyarc.algorithms import M1Algorithm, M2Algorithm, createCARs
import pandas as pd
from sklearn.metrics import accuracy_score

In [11]:
data_train = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/lymph0.csv")
data_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/lymph0.csv")

txns_train = TransactionDB.from_DataFrame(data_train)
txns_test = TransactionDB.from_DataFrame(data_train)

isinstance(txns_test, TransactionDB)



True

In [3]:
from pyarc import CBA, TransactionDB

data_train = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/lymph0.csv")
data_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/lymph0.csv")

txns_train = TransactionDB.from_DataFrame(data_train)
txns_test = TransactionDB.from_DataFrame(data_train)


# get the best association rules
rules = top_rules(txns_train.string_representation)

# convert them to class association rules
cars = createCARs(rules)

classifier = M1Algorithm(cars, txns_train).build()

accuracy = classifier.test_transactions(txns_test)

accuracy

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=19
Rule count: 40504, Iteration: 1
Target rule count satisfied: 1000


0.9615384615384616

In [14]:


directory = "c:/code/python/machine_learning/assoc_rules"

def func(datasetname, unique_transactions=True):
    pd_ds = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/{}.csv".format(datasetname))
    txns = TransactionDB.from_DataFrame(pd_ds, unique_transactions=unique_transactions)
    
    txns_test = TransactionDB.from_DataFrame(pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/{}.csv".format(datasetname)))

    rules = top_rules(txns.string_representation, appearance=txns.appeardict)

    rules.sort(reverse=True)


    cars = createCARs(rules)

    cars.sort(reverse=True)

    if len(cars) > 1000:
        cars = cars[:1000]
        

    print("len(rules)", len(cars))

    m1 = M1Algorithm(cars, txns)
    
    m2 = M2Algorithm(cars, txns)
    
    m1clf = m1.build()
    m2clf = m2.build()
    
    
    actual = list(map(lambda i: i.value, txns_test.class_labels))

    pred = m1clf.predict_all(txns_test)
    predM2 = m2clf.predict_all(txns_test)
    
    accM2 = accuracy_score(predM2, actual)
    acc = accuracy_score(pred, actual)

    return acc, accM2



def mean_func(dataset_name, start=0, end=10, unique_transactions=True):
    files = [ dataset_name + repr(i) for i in range(start, end) ]

    accs = []
    accsM2 = []
    
    for file in files:
        acc, accM2 = func(file, unique_transactions=unique_transactions)
        print("done", file, acc)
        print("done m2", file, accM2)
        accs.append(acc)
        accsM2.append(accM2)
        
    mn = sum(accs) / len(accs)
    mnM2 = sum(accsM2) / len(accsM2)
    
    return mn, mnM2


                
datasets = [
    "iris",
    "breast-w",
    "anneal",
    "hypothyroid",
    "ionosphere",
    "lymph",
    "vehicle",
    "autos",
    "diabetes",
    "glass",
    "heart-h",
    "tic-tac-toe",
    "australian",
    "sick",
    "segment",
    "spambase",
    "sonar",
    "vowel",
    "hepatitis",
    "credit-a",
    "mushroom",
    "house-votes-84",
    "soybean",
    "primary-tumor",
    "credit-g",
    "audiology",
    "breast-cancer",
    "balance-scale",
    "heart-c",
    "kr-vs-kp",
    "pima",
    "heart-statlog"]    

means = []
meansM2 = []
for dataset in datasets[:5]:
    acc, accM2 = mean_func(dataset)
    print("*****")
    print("M1", dataset, acc)
    print("M2", dataset, accM2)
    print("******")
    
    means.append((dataset, acc))
    meansM2.append((dataset, accM2))
    
print("M1")
print(means)
print("\nM2")
print(meansM2)

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=5
Rule count: 96, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=5
Rule count: 166, Iteration: 2
Increasing maxlen 5
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 3
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 4
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 5
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 6
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 3
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 4
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 5
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 6
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000000000004, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 7
Decreasing confidence to 0.25000000000000006
Running apriori with setting: confidence=0.25000000000000006, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 8
Decreasing co

done breast-w1 0.957142857143
done m2 breast-w1 0.957142857143
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=10
Rule count: 741, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=10
Rule count: 4474, Iteration: 2
Target rule count satisfied: 1000
len(rules) 1000
done breast-w2 0.957142857143
done m2 breast-w2 0.957142857143
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=10
Rule count: 683, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=10
Rule count: 4056, Iteration: 2
Target rule count satisfied: 1000
len(rules) 1000
done breast-w3 0.9
done m2 breast-w3 0.9
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=10
Rule count: 708, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, 

done ionosphere0 0.916666666667
done m2 ionosphere0 0.916666666667
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=35
Rule count: 12443, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done ionosphere1 0.916666666667
done m2 ionosphere1 0.916666666667
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=35
Rule count: 11668, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done ionosphere2 0.972222222222
done m2 ionosphere2 0.972222222222
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=35
Rule count: 12162, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done ionosphere3 0.972222222222
done m2 ionosphere3 0.972222222222
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=35
Rule count: 12300, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done ionosphere4 0.91666

In [5]:

m2df = pd.DataFrame(dict(map(lambda i:  (i[0], [i[1]]), meansM2))).transpose()
m2df.columns = ["m2"]

m1df = pd.DataFrame(dict(map(lambda i:  (i[0], [i[1]]), means))).transpose()
m1df.columns = ["m1"]


acc_df = m1df.join(m2df)

acc_df.to_csv("../data/accuracies.csv")

#acc_df = pd.read_csv("../data/accuracies.csv")



In [6]:
acc_df

,m1,m2
lymph,0.788651,0.788651


In [7]:
ARC_accs = pd.read_csv("../data/arc_accs3.csv")

ARC_accs_mod = ARC_accs[["dataset", "accuracy"]].set_index("dataset")

ARC_accs_mod.columns = ["ARC"]

accs = acc_df.join(ARC_accs_mod)

print("M1", accs["m1"].mean())
print("M2", accs["m2"].mean())
print("ARC", accs["ARC"].mean())

M1 0.7886507936507936
M2 0.7886507936507936
ARC 0.81


In [8]:
ARC_accs = pd.read_csv("../data/arc_accs3.csv")

ARC_accs_mod = ARC_accs[["dataset", "accuracy"]].set_index("dataset")

ARC_accs_mod.columns = ["ARC"]

accs = acc_df.join(ARC_accs_mod)

print("M1", accs["m1"].mean())
print("M2", accs["m2"].mean())
print("ARC", accs["ARC"].mean())

M1 0.7886507936507936
M2 0.7886507936507936
ARC 0.81


In [9]:
acc_df

,m1,m2
lymph,0.788651,0.788651


In [10]:
ARC_accs = pd.read_csv("../data/arc_accs3.csv")

ARC_accs_mod = ARC_accs[["dataset", "accuracy"]].set_index("dataset")

ARC_accs_mod.columns = ["ARC"]

accs = acc_df.join(ARC_accs_mod)

print("M1", accs["m1"].mean())
print("M2", accs["m2"].mean())
print("ARC", accs["ARC"].mean())

M1 0.7886507936507936
M2 0.7886507936507936
ARC 0.81


In [11]:
printaccs = accs[["m1", "m2", "ARC"]] * 100
printaccs = printaccs[["m1", "m2", "ARC"]].round(1)

In [12]:
printaccs.to_csv("../data/all_accuracies.csv")

In [13]:
"""
M1 - difference between unique and non-unique transactions
"""

means_unique = []
meansM2_unique = []
for dataset in datasets[:5]:
    acc, accM2 = mean_func(dataset)
    print("*****")
    print("M1", dataset, acc)
    print("M2", dataset, accM2)
    print("******")
    
    means_unique.append((dataset, acc))
    meansM2_unique.append((dataset, accM2))
    
    
means_nonunique = []
meansM2_nonunique = []
for dataset in datasets[:5]:
    acc, accM2 = mean_func(dataset, unique_transactions=False)
    print("*****")
    print("M1", dataset, acc)
    print("M2", dataset, accM2)
    print("******")
    
    means_nonunique.append((dataset, acc))
    meansM2_nonunique.append((dataset, accM2))

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=5
Rule count: 96, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=5
Rule count: 166, Iteration: 2
Increasing maxlen 5
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 3
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 4
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 5
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 6
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000

Running apriori with setting: confidence=0.30000000000000004, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 7
Decreasing confidence to 0.25000000000000006
Running apriori with setting: confidence=0.25000000000000006, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 8
Decreasing confidence to 0.20000000000000007
Running apriori with setting: confidence=0.20000000000000007, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 9
Decreasing confidence to 0.15000000000000008
Running apriori with setting: confidence=0.15000000000000008, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 10
Decreasing confidence to 0.10000000000000007
Running apriori with setting: confidence=0.10000000000000007, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 145, Iteration: 11
Decreasing confidence to 0.05000000000000007
Running apriori with setting: confidence=0.05000000000000007, support=0.0

len(rules) 1000
done breast-w1 0.957142857143
done m2 breast-w1 0.957142857143
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=10
Rule count: 741, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=10
Rule count: 4474, Iteration: 2
Target rule count satisfied: 1000
len(rules) 1000
done breast-w2 0.957142857143
done m2 breast-w2 0.957142857143
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=10
Rule count: 683, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=10
Rule count: 4056, Iteration: 2
Target rule count satisfied: 1000
len(rules) 1000
done breast-w3 0.9
done m2 breast-w3 0.9
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=10
Rule count: 708, Iteration: 1
Increasing maxlen 4
Running apriori with setting: 

KeyboardInterrupt: 

In [13]:
set(frozenset([1, 2]))

{1, 2}